# World Bank - Commodity Prices (Pink Sheet)

World Bank Commodity Markets ("Pink Sheet") data provides monthly commodity price data.

Data source: https://www.worldbank.org/en/research/commodity-markets

API Documentation: https://datahelpdesk.worldbank.org/knowledgebase/topics/125589

## Python setup

In [1]:
# system imports
from pathlib import Path
from typing import Any, Dict, List
import time
import requests
from datetime import datetime, timedelta
import io

In [2]:
# analytic imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display

# Note: pandas requires openpyxl to read Excel files
# Install if needed: pip install openpyxl

In [3]:
# local imports
import mgplot as mg

In [4]:
# plotting setup
CHART_DIR = "./CHARTS/WB-COMMODITIES/"
mg.set_chart_dir(CHART_DIR)
mg.clear_chart_dir()
SHOW = False
SOURCE = "World Bank Commodity Markets (Pink Sheet)"

## World Bank Data Configuration

World Bank distributes commodity price data via Excel files (not API)

In [5]:
# World Bank Commodity Price Data Excel URL
# The Pink Sheet data is distributed as Excel files, updated monthly

WB_MONTHLY_DATA_URL = "https://thedocs.worldbank.org/en/doc/18675f1d1639c7a34d463f59263ba0a2-0050012025/related/CMO-Historical-Data-Monthly.xlsx"
WB_ANNUAL_DATA_URL = "https://thedocs.worldbank.org/en/doc/18675f1d1639c7a34d463f59263ba0a2-0050012025/related/CMO-Historical-Data-Annual.xlsx"

print("📊 World Bank Commodity Price Data (Pink Sheet)")
print(f"Data source: {WB_MONTHLY_DATA_URL}")
print("\nNote: Data is downloaded directly from Excel files - no API key required!")

📊 World Bank Commodity Price Data (Pink Sheet)
Data source: https://thedocs.worldbank.org/en/doc/18675f1d1639c7a34d463f59263ba0a2-0050012025/related/CMO-Historical-Data-Monthly.xlsx

Note: Data is downloaded directly from Excel files - no API key required!


## Data Retrieval Functions

In [6]:
def download_wb_commodity_data(url: str) -> pd.DataFrame:
    """Download World Bank commodity price data from Excel file."""
    print(f"📥 Downloading data from World Bank...")
    
    try:
        response = requests.get(url, timeout=60)
        response.raise_for_status()
        print(f"   ✅ Downloaded {len(response.content):,} bytes")
        
        excel_file = io.BytesIO(response.content)
        xls = pd.ExcelFile(excel_file)
        
        return xls
        
    except Exception as e:
        print(f"   ❌ Error downloading data: {e}")
        return None


def expand_commodity_name(name: str) -> str:
    """Expand common commodity abbreviations to full names."""
    expansions = {
        'DAP': 'Diammonium Phosphate (Fertilizer)',
        'TSP': 'Triple Superphosphate (Fertilizer)',
        'Urea': 'Urea (Fertilizer)',
        'Potassium chloride': 'Potassium Chloride (Fertilizer)',
    }
    
    # Try exact match first (stripping whitespace)
    name_clean = name.strip()
    if name_clean in expansions:
        return expansions[name_clean]
    
    # Try partial match (for cases like "Potassium chloride **")
    for key, full_name in expansions.items():
        if name_clean.startswith(key):
            # Preserve any suffix like " **"
            suffix = name_clean[len(key):]
            return full_name + suffix
    
    return name


def parse_wb_monthly_data(xls: pd.ExcelFile, start_year: int = 2010) -> Dict[str, pd.Series]:
    """Parse World Bank monthly commodity price data from Excel file."""
    commodity_data = {}
    data_sheet = 'Monthly Prices'
    
    if data_sheet not in xls.sheet_names:
        print(f"❌ Could not find '{data_sheet}' sheet")
        return {}
    
    try:
        # Read header rows to get commodity names and units
        df_header = pd.read_excel(xls, sheet_name=data_sheet, header=None, nrows=15)
        
        # Find data start row (looking for dates like "1960M01")
        data_start_row = None
        for i in range(len(df_header)):
            val = str(df_header.iloc[i, 0])
            if 'M' in val and len(val) >= 7:
                parts = val.split('M')
                if len(parts) == 2 and parts[0].isdigit() and parts[1].isdigit():
                    data_start_row = i
                    break
        
        if data_start_row is None:
            print("   ❌ Could not find data start row")
            return {}
        
        # Extract commodity names (row 4) and units (row 5)
        commodity_names = df_header.iloc[4, :].values
        units = df_header.iloc[5, :].values
        
        # Read the actual data
        df = pd.read_excel(xls, sheet_name=data_sheet, header=None, skiprows=data_start_row)
        df.columns = commodity_names
        
        # Parse dates from "1960M01" format to PeriodIndex
        dates = []
        for date_str in df.iloc[:, 0].astype(str):
            try:
                if 'M' in date_str:
                    parts = date_str.split('M')
                    year, month = int(parts[0]), int(parts[1])
                    dates.append(pd.Period(year=year, month=month, freq='M'))
                else:
                    dates.append(pd.NaT)
            except:
                dates.append(pd.NaT)
        
        df.index = pd.PeriodIndex(dates, freq='M')
        df = df[df.index.notna()]
        
        # Filter by start year
        df = df[df.index.year >= start_year]
        
        print(f"   📊 Loaded {df.shape[1]-1} commodities, {len(df)} observations from {start_year}")
        
        # Convert each column to a Series
        for i in range(1, len(df.columns)):
            col_name = commodity_names[i]
            unit = units[i] if i < len(units) else ''
            
            if pd.isna(col_name) or str(col_name).strip() == '' or str(col_name) == 'nan':
                continue
            
            series = pd.to_numeric(df.iloc[:, i], errors='coerce').dropna()
            
            if len(series) >= 10:
                series_name = expand_commodity_name(str(col_name).strip())
                if pd.notna(unit) and str(unit).strip() and str(unit) != 'nan':
                    series_name += f" {str(unit).strip()}"
                
                series.name = series_name
                commodity_data[series_name] = series
        
        print(f"   ✅ Successfully loaded: {len(commodity_data)} commodity series")
        
        return commodity_data
        
    except Exception as e:
        print(f"\n❌ Error parsing Excel data: {e}")
        import traceback
        traceback.print_exc()
        return {}

In [7]:
def get_all_wb_commodities(url: str, start_year: int = 2010) -> Dict[str, pd.Series]:
    """
    Download and parse all World Bank commodity price data.
    
    Args:
        url: URL to the Excel file
        start_year: Starting year for data
    
    Returns:
        Dictionary mapping commodity names to price series
    """
    print("="*80)
    print("DOWNLOADING WORLD BANK COMMODITY PRICE DATA")
    print("="*80)
    
    # Download the Excel file
    xls = download_wb_commodity_data(url)
    
    if xls is None:
        return {}
    
    # Parse the monthly data
    commodity_data = parse_wb_monthly_data(xls, start_year=start_year)
    
    print("\\n" + "="*80)
    print("DATA COLLECTION COMPLETE")
    print("="*80)
    
    return commodity_data

## Data Collection

In [8]:
# Collect commodity price data from World Bank
print("💰 COLLECTING COMMODITY PRICE DATA FROM WORLD BANK")
print("Downloading Pink Sheet Excel file...")
print("Note: No API key required!\\n")

# Get commodity price data from 2010 onwards
commodity_prices = get_all_wb_commodities(WB_MONTHLY_DATA_URL, start_year=2010)

if commodity_prices:
    print(f"\\n📊 DATA OVERVIEW")
    print(f"   Commodities collected: {len(commodity_prices)}")
    
    print("\\n📅 DATA RANGE FOR EACH SERIES:")
    for name, series in list(commodity_prices.items())[:10]:  # Show first 10
        start_date = series.index[0]
        end_date = series.index[-1]
        obs = len(series)
        print(f"   {name[:50]:<50} : {start_date} to {end_date} ({obs} obs)")
    
    if len(commodity_prices) > 10:
        print(f"   ... and {len(commodity_prices) - 10} more series")
else:
    print("\\n⚠️  No data was successfully collected!")

💰 COLLECTING COMMODITY PRICE DATA FROM WORLD BANK
Note: No API key required!\n
DOWNLOADING WORLD BANK COMMODITY PRICE DATA
📥 Downloading data from World Bank...
   ✅ Downloaded 777,464 bytes
   📊 Loaded 71 commodities, 190 observations from 2010
   ✅ Successfully loaded: 71 commodity series
\n================================================================================
DATA COLLECTION COMPLETE
\n📊 DATA OVERVIEW
   Commodities collected: 71
\n📅 DATA RANGE FOR EACH SERIES:
   Crude oil, average ($/bbl)                         : 2010-01 to 2025-10 (190 obs)
   Crude oil, Brent ($/bbl)                           : 2010-01 to 2025-10 (190 obs)
   Crude oil, Dubai ($/bbl)                           : 2010-01 to 2025-10 (190 obs)
   Crude oil, WTI ($/bbl)                             : 2010-01 to 2025-10 (190 obs)
   Coal, Australian ($/mt)                            : 2010-01 to 2025-10 (190 obs)
   Coal, South African ** ($/mt)                      : 2010-01 to 2025-10 (190 obs)
   Natural 

## Plot Individual Commodities

In [9]:
# Plot each commodity
if commodity_prices:
    for name, series in commodity_prices.items():
        ax = mg.line_plot(series, width=2, color='darkblue', annotate=True,)
        
        # Check if this commodity has ** marker
        lfooter_text = f"{len(series)} observations. Latest: {series.index[-1]}. "
        if '**' in name:
            lfooter_text += "** = series replacements andor methodology changes. "
        
        mg.finalise_plot(
            ax,
            title=name,
            ylabel="Price (USD)",
            xlabel=None,
            rfooter=SOURCE,
            lfooter=lfooter_text,
            show=SHOW,
        )

## Notes

### World Bank Commodity Price Data

The World Bank maintains the "Pink Sheet" - a comprehensive monthly commodity price dataset covering:
- Energy commodities (crude oil, natural gas, coal)
- Metals and minerals (base metals, precious metals)
- Agriculture (grains, oilseeds, food)
- Fertilizers
- Raw materials

**Data Characteristics:**
- Frequency: Monthly
- Currency: Nominal US dollars (unless otherwise specified)
- Historical depth: Many series go back to 1960
- No API key required!

**Data Source:**
- Excel files distributed monthly by World Bank
- Direct download URL: https://thedocs.worldbank.org/en/doc/.../CMO-Historical-Data-Monthly.xlsx
- Updated on the second business day of each month

**Data Structure:**
- Excel file with multiple sheets
- Main data typically in 'Monthly Prices' sheet
- First column: dates
- Remaining columns: individual commodity prices

## The End

In [10]:
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda

Last updated: 2025-11-18 08:16:46

Python implementation: CPython
Python version       : 3.14.0
IPython version      : 9.6.0

conda environment: n/a

Compiler    : Clang 20.1.4 
OS          : Darwin
Release     : 25.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

pathlib   : 1.0.1
pandas    : 2.3.3
requests  : 2.32.5
numpy     : 2.3.4
mgplot    : 0.2.12
typing    : 3.10.0.0
matplotlib: 3.10.7
IPython   : 9.6.0

Watermark: 2.5.0

